In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import joblib

In [ ]:
#Read data
df = pd.read_csv('/home/edward/Homelab/Machine_learning/Drug_prediction/Data/drugsComTest_complete.csv', sep=',')


input_c = input("Enter the medical condition: ")

#Create mask variable to define and process data to required parameters, in this case to keep all values with rating 10 as True. The mask results categorize as True or False, and keeps all True. Apply the mask to the DataFrame and save it as a new DataFrame.
mask = df["rating"] == 10
df_mask = df[mask]

mask_2 = df_mask["condition"] == input_c
df_mask = df_mask[mask_2]


#Extract specific data for independent variable within the DataFrame columns.
X = df_mask[["usefulCount", "review"]]

#Replace name with numbers, by creating labels for the independent variable "condition".
#Import Sklearn LabelEncoder to encode the independent variable "condition".
from sklearn import preprocessing

#Extract specific data for dependent variable within the DataFrame columns.
Y = df_mask[['drugName']]

#Replace name with numbers - Labels
le_drugN = preprocessing.LabelEncoder()
le_drugN.fit(Y['drugName'])
Y['drugName'] = le_drugN.transform(Y['drugName'])

In [ ]:
#Get only the top 20 drugNames with the highest number of usefulCount.

top_20 = X.sort_values(by=['usefulCount'], ascending=False)
top_20 = top_20.head(20)

#Get the index of the top 20 drugNames.
top_20_index = top_20.index

top_20_index_list = top_20_index.tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
def sentiment_analysis(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
#Create a new column in the DataFrame, with the sentiment analysis of the review.
from cgi import print_environ


sentiment_score = []

#Sentiment analysis for the top 20 drugNames.
for i in top_20_index_list:
    sentiment_score.append(sentiment_analysis(X['review'][i]))

print(sentiment_score)


In [ ]:
#Indexes of the highest scores in sentiment_score.
highest_score_index = [i for i, x in enumerate(sentiment_score) if x == max(sentiment_score)]

recommend_drug_index = []
recommend_drug_name = []
recommend_drug_score = []

for k in highest_score_index:
    recommend_drug_index.append(top_20_index_list[k])

for o in recommend_drug_index:
    recommend_drug_name.append(Y['drugName'][o])

#Transform the drugName numbers in recommend_drug_name back to the original names.
recommend_drug_name = le_drugN.inverse_transform(recommend_drug_name)

print(recommend_drug_name)

input_review = input("Enter the index of the recommended drug you want to have the review: ")

client_request_drug_index = recommend_drug_index[int(input_review)]

#get the review of the drug that the client requested.
client_request_drug_review = X['review'][client_request_drug_index]

print(client_request_drug_review)

